In [1]:
from utils import * 
import numpy as np

URL = "http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz"
f_name = "lastfm-dataset-1K.tar.gz"
dir_name = "lastfm-dataset-1K"
dataset_f_name = "userid-timestamp-artid-artname-traid-traname.tsv"

In [2]:
download_dataset(URL, f_name)

Dataset Exists...


In [3]:
user_data = load_dataset(dir_name, dataset_f_name, 1000000)

Read File...


In [4]:
user_data[:10]

,userid,timestamp,artist-id,artist-name,track-id,track-name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Improvisation (Live_2009_4_15)
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Glacier (Live_2009_4_15)
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Parolibre (Live_2009_4_15)
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Bibo No Aozora (Live_2009_4_15)


## Cleaning infrequent data

First remove infrequent tracks (<10 plays)

In [5]:
# Removing infrequent tracks.
if user_data['track-id'].isnull().sum() > 0:
    user_data = user_data.dropna(axis = 0, subset = ['track-id'])

In [6]:
user_data.shape

(865616, 6)

In [7]:
total_plays = user_data.groupby(["track-id"]).size().reset_index()
total_plays.rename(columns = {0: 'total'}, inplace = True)
total_plays.describe()

,total
count,126986.000000
mean,6.816625
std,25.337567
min,1.000000
25%,1.000000
50%,2.000000
75%,6.000000
max,2069.000000


In [8]:
frequent_plays = total_plays[total_plays['total'] >= 10]['track-id']
frequent_plays.describe()

count                                    20666
unique                                   20666
top       8837c142-45e4-495f-abd9-4f3c016dfa52
freq                                         1
Name: track-id, dtype: object

In [9]:
# Drop infrequent tracks.
data = user_data[user_data['track-id'].isin(frequent_plays)]
data.describe()

,userid,timestamp,artist-id,artist-name,track-id,track-name
count,595039,595039,595039,595039,595039,595039
unique,41,592738,3331,3309,20666,19328
top,user_000033,2008-11-24T11:38:21Z,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,82558949-cd98-4c58-af35-3f1a9430d52e,Heartless
freq,58772,24,26616,26616,2069,2120


Now, remove infrequent users.

In [10]:
# Removing infrequent tracks.
if data['userid'].isnull().sum() > 0:
    data = data.dropna(axis = 0, subset = ['userid'])

In [11]:
data.shape

(595039, 6)

In [12]:
user_plays = data.groupby(["userid"]).size().reset_index()
user_plays.rename(columns = {0: 'total'}, inplace = True)
user_plays.describe()

,total
count,41.000000
mean,14513.146341
std,15059.907282
min,176.000000
25%,3090.000000
50%,10437.000000
75%,18144.000000
max,58772.000000


In [13]:
frequent_users = user_plays[user_plays['total'] >= 10]['userid']
frequent_users.describe()

count              41
unique             41
top       user_000035
freq                1
Name: userid, dtype: object

In [14]:
# Drop infrequent tracks.
data = data[data['userid'].isin(frequent_users)]
data.describe()

,userid,timestamp,artist-id,artist-name,track-id,track-name
count,595039,595039,595039,595039,595039,595039
unique,41,592738,3331,3309,20666,19328
top,user_000033,2008-11-24T11:38:21Z,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,82558949-cd98-4c58-af35-3f1a9430d52e,Heartless
freq,58772,24,26616,26616,2069,2120


In [15]:
user_play_counts=data.groupby(["userid"]).size().to_frame('size').reset_index()
user_play_counts

,userid,size
0,user_000001,6723
1,user_000002,42930
2,user_000003,10437
3,user_000004,8200
4,user_000005,13211
5,user_000006,10246
6,user_000007,182
7,user_000008,34997
8,user_000009,1857
9,user_000010,3090


In [18]:
groups=data.groupby('userid')
groups.groups

{'user_000001': Int64Index([   10,    12,    14,    15,    74,   174,   255,   256,   257,
               258,
             ...
             16653, 16654, 16655, 16659, 16667, 16669, 16676, 16682, 16683,
             16684],
            dtype='int64', length=6723),
 'user_000002': Int64Index([16685, 16686, 16696, 16697, 16698, 16704, 16705, 16708, 16710,
             16711,
             ...
             74112, 74113, 74114, 74115, 74117, 74118, 74119, 74120, 74121,
             74122],
            dtype='int64', length=42930),
 'user_000003': Int64Index([74127, 74128, 74144, 74147, 74161, 74163, 74164, 74165, 74166,
             74167,
             ...
             93602, 93603, 93605, 93606, 93611, 93612, 93613, 93614, 93615,
             93616],
            dtype='int64', length=10437),
 'user_000004': Int64Index([ 93622,  93627,  93635,  93637,  93638,  93639,  93648,  93649,
              93650,  93651,
             ...
             112007, 112008, 112010, 112016, 112019, 112020, 1

In [ ]:
import datetime, dateutil.parser
sessions = []
for user, indices in groups.groups.iteritems():
    print("------------------------------------------ User ",user)
    user_record = data.loc[indices, :].sort_values(['timestamp'])
    print user_record.shape
    session = [user_record.iloc[0].name]
    for i in range(0, user_record.shape[0] - 1):
        d1 = int(dateutil.parser.parse(user_record.iloc[i]['timestamp']).strftime('%s'))
        d2 = int(dateutil.parser.parse(user_record.iloc[i+1]['timestamp']).strftime('%s'))
        if(d2-d1 < 800 ):
            session.append(user_record.iloc[i+1].name)
        else:
            if(len(session)>10):
                print(session)
                sessions.append(session)
            session=[]
            session.append(user_record.iloc[i+1].name)

('------------------------------------------ User ', 'user_000038')
(7167, 6)
[958710, 958709, 958708, 958707, 958706, 958705, 958704, 958703, 958702, 958701, 958700]
[958668, 958667, 958666, 958665, 958664, 958663, 958662, 958661, 958660, 958659, 958658, 958657]
[958494, 958493, 958492, 958491, 958490, 958489, 958488, 958487, 958486, 958485, 958484]
[958462, 958461, 958460, 958459, 958458, 958457, 958456, 958455, 958454, 958453, 958452]
[958413, 958412, 958411, 958410, 958409, 958408, 958407, 958406, 958405, 958404, 958403, 958402]
[958287, 958286, 958285, 958284, 958283, 958282, 958281, 958280, 958279, 958278, 958277, 958276, 958275, 958274]
[958238, 958237, 958236, 958235, 958234, 958233, 958232, 958231, 958230, 958229, 958228]
[958095, 958094, 958093, 958092, 958091, 958090, 958089, 958088, 958087, 958086, 958085]
[958065, 958064, 958063, 958062, 958061, 958060, 958059, 958058, 958057, 958056, 958055, 958054]
[958040, 958039, 958038, 958037, 958036, 958035, 958034, 958033, 958032, 

## Storing id-name mappings

Forming an artist, track id-name mapping.

In [ ]:
artists = data[["artist-id", "artist-name"]].drop_duplicates()


In [ ]:
track = data[["track-id", "track-name"]].drop_duplicates()

Forming a mapping from artist to track

In [ ]:
artist_track = data[["track-id", "artist-id"]].drop_duplicates()

## Playcount matrix

In [ ]:
playcount = data.groupby(["userid", "track-id"]).size().reset_index()

In [ ]:
playcount.rename(columns = {0: 'playcount'}, inplace = True)

In [ ]:
playcount[:10]

In [ ]:
from scipy.sparse import csr_matrix

playcounts = playcount.pivot(index = 'userid', columns = 'track-id', values = 'playcount').fillna(0)


In [ ]:
plays_matrix = csr_matrix(playcounts.values)
playcounts[:10]

## Conversion of Playcounts to Ratings

Based on work in A hybrid online-product recommendation system: Combining implicit
rating-based collaborative filtering and sequential pattern analysis by Choi et al.

Absolute Preference: AP(u, i) = ln((n(u played i)/n(u played anything)) + 1)

Relative Preference: RP(u, i) = AP(u,i) / Max_{c \in U} AP(c, i)

Implicit Rating: R(u, i) = Round up(5 * RP (u,i))

In [ ]:
# List of number of songs ever played by each user. Reduces computation
user_plays = plays_matrix.sum(axis=1) # Sum all columns
print(user_plays[:20])
print(user_plays.shape)

#number_of_users

In [ ]:
# Compute Absolute Rating
absolute_preferences = np.log(plays_matrix / user_plays + 1)

In [ ]:
absolute_preferences.shape
number_of_users=absolute_preferences.shape[0]
number_of_items=absolute_preferences.shape[1]

In [ ]:
# 2. List of maximum ratings of each item
max_ratings = absolute_preferences.max(axis=0) # Along columns
print(max_ratings[:10])
print(max_ratings.shape)

In [ ]:
print(max_ratings[max_ratings ==0].size)

In [ ]:
print(user_plays[user_plays == 0].size)

In [ ]:
print(max_ratings.shape)

In [ ]:
# Relative Preference: RP(u, i) = AP(u,i) / Max_{c \in U} AP(c, i)
relative_preferences = absolute_preferences / (max_ratings)
print(absolute_preferences.shape)
print(relative_preferences.shape)

In [ ]:
# Implicit Rating: R(u, i) = Round up(5 * RP (u,i))
implicit_ratings = np.ceil(5 * relative_preferences)

In [ ]:
implicit_ratings.max(axis=1)
print(implicit_ratings[:10, :10])

In [ ]:
implicit_ratings.shape

## Metrics

We use the following metrics

### For Rating Prediction

#### Mean Absolute Error

#### Root Mean Square Error

### For Top-n Recommendations

#### Precision at k

#### Mean Average Precision at k

#### Truncated Normalized Discounted Cumulative Gain (NDCG@K)

In [ ]:
from metrics import * 

In [ ]:
def compute_biases(ratings):
    """computes biases for every user and every item, and the overall average of the ratings
    
    input:
        ratings: the matrix of ratings
    output: 
        mu: overall average of ratings
        user_biases: deviations of user means wrt mu, shape (1,N)
        item_biases: deviations of item means wrt mu, shape (D,1)
    """
    num_users, num_items = ratings.shape
    

    # boolean array showing the nonzero entries
    nz = ratings != 0

    # the mean is computed over ALL nonzero ratings
    mu = ratings[nz].mean()
    print(num_users)
    print(num_items)
    # mean over all nonzero ratings
    # biases for every user/item
    item_means = ratings.sum(axis=0) / (nz).sum(axis=0)
    user_means = ratings.sum(axis=1) / (nz).sum(axis=1)
    user_biases = np.reshape(user_means - mu, (1, num_users))
    item_biases = np.reshape(item_means - mu, (num_items, 1))
    return mu, user_biases, item_biases




In [ ]:
mu, user_biases, item_biases  = compute_biases(implicit_ratings)

In [ ]:
def prediction_biased(W, Z, mu, user_biases, item_biases):
    """ returns the biased prediction matrix.
    
    input :
        W: item feature matrix
        Z: user feature matrix
        mu: average of all ratings
        user_biases: deviations of user means wrt mu, shape (1,N)
        item_biases: deviations of item means wrt mu, shape (D,1) 
    output:
        X_hat : has shape (D, N)
    """
    # compute the biased prediction matrix
    X_hat = (W.transpose().dot(Z)) + item_biases + user_biases + mu)

    # make sure that ratings stay within valid range of 1-5
    if np.isscalar(X_hat):
        if X_hat < 1:
            X_hat = 1
        elif X_hat > 5:
            X_hat = 5
    else:
        X_hat[X_hat < 1] = 1.
        X_hat[X_hat > 5] = 5.

    return X_hat